In [1]:
%useLatestDescriptors
%use dataframe, lets-plot

Kotlin DataFrame can be used to work with REST APIs, JSON (let's say some logs or dump of a document database) and other hierarchical structures without the need to write boilerplate code that describes your data.  

Let's see how it works. This example is divided into two sections.
- Section 1. Answer these questions using a prepared data sample:
1. How many contributors there are and how much do they contribute?
2. How many of those contributors are not JetBrains members (according to GitHub)?
3. What is the most starred project?
- Section 2. Build the data sample 

Section 1.

In [2]:
val organizationsRaw = DataFrame.read("https://raw.githubusercontent.com/Kotlin/dataframe/master/data/jetbrains.json")
organizationsRaw

DataFrame: rowsCount = 1, columnsCount = 4

In [3]:
organizationsRaw.schema()

url: String
name: String
repos: *
    id: String
    node_id: String
    name: String
    full_name: String
    private: String
    html_url: String
    description: String?
    fork: String
    url: String
    created_at: String
    updated_at: String
    pushed_at: String
    homepage: String?
    size: String
    stargazers_count: String
    watchers_count: String
    language: String?
    has_issues: String
    has_projects: String
    has_downloads: String
    has_wiki: String
    has_pages: String
    forks_count: String
    archived: String
    disabled: String
    open_issues_count: String
    allow_forking: String
    is_template: String
    topics: String
    visibility: String
    forks: String
    open_issues: String
    watchers: String
    default_branch: String
    permissions:
        admin: String
        maintain: String
        push: String
        triage: String
        pull: String
    contributors: *
        login: String
        id: String
        node_id: String

In [ ]:
val organizations = organizationsRaw.parse()
organizations

In [5]:
organizations.schema()

url: URL
name: String
repos: *
    id: Int
    node_id: String
    name: String
    full_name: String
    private: Boolean
    html_url: URL
    description: String?
    fork: Boolean
    url: URL
    created_at: String
    updated_at: String
    pushed_at: String
    homepage: String?
    size: Int
    stargazers_count: Int
    watchers_count: Int
    language: String?
    has_issues: Boolean
    has_projects: Boolean
    has_downloads: Boolean
    has_wiki: Boolean
    has_pages: Boolean
    forks_count: Int
    archived: Boolean
    disabled: Boolean
    open_issues_count: Int
    allow_forking: Boolean
    is_template: Boolean
    topics: String
    visibility: String
    forks: Int
    open_issues: Int
    watchers: Int
    default_branch: String
    permissions:
        admin: Boolean
        maintain: Boolean
        push: Boolean
        triage: Boolean
        pull: Boolean
    contributors: *
        login: String
        id: Int
        node_id: String
        gravatar_id: S

This data is indeed hierarchical. organizations is itself a dataframe. It has a child column `repos`, a dataframe with lots of data.

1. **How many contributors there are?**

The easiest way to work with hierarchical data is to pick only required data from it. It is nice to have a huge dataframe with all relations between data, but carrying it around involves extra complexity. So let's start by getting contributors from inside of the data structure.

In [6]:
val jb = organizations[0]

In [7]:
val jbRepos = jb.repos

In [8]:
jbRepos.contributors

... showing only top 20 of 100 rows DataColumn [100]

So `contributors` is a [FrameColumn](https://kotlin.github.io/dataframe/datacolumn.html#framecolumn), a column of dataframes. There is a family of operations [to work with several dataframes](https://kotlin.github.io/dataframe/multipledataframes.html). We can use [`concat`](https://kotlin.github.io/dataframe/concat.html) to simplify the structure.

In [9]:
val jbContributors = jbRepos.contributors.concat()
jbContributors

... showing only top 20 of 1673 rows DataFrame [1673 x 8]

Probably not all of them are unique rows. Let's use [`distinctBy`](https://kotlin.github.io/dataframe/distinct.html#distinctby)

In [10]:
jbContributors.distinctBy { login }.count()

1123

So, the answer to the question "How many contributors there are" is 1123.

Now that we have a dataframe with all contributors from jb repositories, let's see **how much do they contribute** with the help of [groupBy aggregations](https://kotlin.github.io/dataframe/groupby.html#aggregation)

In [11]:
val top = jbContributors.groupBy { login and url }.sum { contributions }.sortByDesc { contributions }

2. How many of those contributors are not JetBrains members (according to GitHub)?

In [12]:
val memberIds = jb.members.id.toSet()
memberIds.size

187

In [13]:
val communityContributors = jbContributors.filter { it.id !in memberIds }

In [14]:
communityContributors.groupBy { login and url }.sum { contributions }.sortByDesc { contributions }

... showing only top 20 of 997 rows DataFrame [997 x 3]

Let's visualise the difference two groups of contributors: members and non-members. We will explain `map` operation in more details later

In [15]:
val df = jbRepos.mapToFrame { 
    "id" from { id }
    +name
    "total" from  { contributors.count() }
    "community" from { contributors.filter { it.id !in memberIds }.count() }
}
df

... showing only top 20 of 100 rows DataFrame [100 x 4]

In [16]:
letsPlot { 
    x = df.name.values() 
    y = "contrib"
} + ggsize(1200, 500) + geomStep(
    data = mapOf("contrib" to df.total.values()),
    position = Pos.stack
) + geomStep(
    data = mapOf("contrib" to df.community.values()),
    color = "dark_green",
)

3. **What is the most starred project?** Naturally, as our dataframe has only one row, we can find it out like this

In [17]:
jbRepos.maxBy { stargazers_count }

DataRow [37]

But usually dataframe has several rows. With dataframe, you can work with hierarchical data structures as a whole, if you need to. Let's use `map` to find a repository with the most stars for each row in the `jb` dataframe. The result of `map` is a data column `most_starred_repo`. 

In [18]:
organizations.mapToColumn("most_starred_repo") {
    it.repos.maxBy { stargazers_count }
}

DataFrame [1 x 37]

Expressions inside map, a row expression (docs), enable two different syntax to refer to data. **You can write `this.repos`, `it.repos` or just `repos`. It's all the same, but let's use shorter syntax from now on.**

In [19]:
organizations.mapToColumn("most_starred_repo") {
    // Alternative syntax
    // this.repos.maxBy { stargazers_count }
    // it.repos.maxBy { stargazers_count }
    repos.maxBy { stargazers_count }
}

DataFrame [1 x 37]

To make the result more obvious, let's create a new dataframe with a subset of columns from repos using `select`. Then we will use `head` to check the first 5 rows in the dataframe. 

In [20]:
organizations[0].repos.select { name and html_url and stargazers_count }.head(5)

DataFrame [5 x 3]

Looks better, doesn't it? Let's combine two previous examples

In [21]:
organizations.mapToColumn("most_starred_repo") {
    repos.select { name and html_url and stargazers_count }.maxBy { stargazers_count }
}

DataFrame [1 x 3]

3a) What is **the most and the least** starred project? There is a special syntax for `map` when you need to compute more than one column. This `map` creates a new dataframe.

In [22]:
organizations.mapToFrame { 
    "most_starred_repo" from { 
        repos
            .select { name and html_url and stargazers_count }
            .maxBy { stargazers_count } 
    }
    "least_starred_repo" from { 
        repos
            .select { name and html_url and stargazers_count }
            .minBy { stargazers_count } 
    }
}

DataFrame [1 x 2]

We can observe that we have to write the same select twice. Instead of this, let's **create a new dataframe, where column `repos`  has only 3 child columns: `name and html_url and stargazers_count`** and then get the most and least starred project again.

Here we want to modify values inside the dataframe, and the documentation suggests we can use [`update`](https://kotlin.github.io/dataframe/update.html) for this task.

In [23]:
val organizationsWithMinifiedRepos = organizations.update {
    // We want to change repos, so pick this column
    repos 
}
.with { 
    // Here *it* is *repos*, and we can use `select` as we did in the previous example.
    // Value of the `repos` column will be replaced by the result of the following expression
    it.select { name and html_url and stargazers_count } 
}   

Note that **dataframe is an immutable structure** and operators do not change values inside original dataframe. 

In [24]:
println(organizations.repos[0].columnNames())
println()
println(organizationsWithMinifiedRepos.repos[0].columnNames())

[id, node_id, name, full_name, private, html_url, description, fork, url, created_at, updated_at, pushed_at, homepage, size, stargazers_count, watchers_count, language, has_issues, has_projects, has_downloads, has_wiki, has_pages, forks_count, archived, disabled, open_issues_count, allow_forking, is_template, topics, visibility, forks, open_issues, watchers, default_branch, permissions, contributors, license]

[name, html_url, stargazers_count]


Here is the answer for **what are the most and the least starred projects**.

In [25]:
organizationsWithMinifiedRepos.mapToFrame { 
    "most_starred_repo" from { 
        repos.maxBy { stargazers_count } 
    }
    "least_starred_repo" from { 
        repos.minBy { stargazers_count } 
    }
}

DataFrame [1 x 2]

Expressions inside map, the [**row expressions**](https://kotlin.github.io/dataframe/datarow.html#row-expressions), can be as complex as your fantasy goes. For example, we can pick the five most starred and five least starred repositories.

In [26]:
organizationsWithMinifiedRepos.mapToColumn("interesting_repos") {
    val sorted = repos.sortByDesc { stargazers_count }
    sorted.take(5).concat(sorted.takeLast(5))
}

DataColumn [1]

That's it. We used the following operations to explore our data sample:
* [`map`](https://kotlin.github.io/dataframe/map.html) to create a data column and dataframe with values computed from rows of original dataframe
* [`select`](https://kotlin.github.io/dataframe/columnselectors.html) to create dataframe with subset of columns from original
* [`convert`](https://kotlin.github.io/dataframe/convert.html) to create dataframe with changed values in some columns (in our case it was `repos`)

Here is some code required to work with GitHub API with API key. Without API the key, there is a limit of 30 requests or so. 
Follow the instruction to get a personal access token: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token. Then update line `val credentials: Credentials? = null` like `val credentials: Credentials? = Credentials("username", "your_token")`

In [27]:
@file:DependsOn("org.http4k:http4k-core:4.16.3.0")
@file:DependsOn("org.http4k:http4k-client-apache:4.16.3.0")

In [28]:
import org.http4k.client.ApacheClient
import org.http4k.core.Method.GET
import org.http4k.core.Request

val client = ApacheClient()

In [29]:
data class Credentials(val name: String, val token: String)

In [30]:
import java.util.Base64

fun authHeader(name: String, token: String): String {
    return "Basic " + Base64.getEncoder().encode("$name:$token".toByteArray()).toString(Charsets.UTF_8)
}

fun readDf(url: String, credentials: Credentials?): DataFrame<*> {
    val request = Request(GET, url).let {
        if (credentials != null) {
            it.header("Authorization", authHeader(credentials.name, credentials.token))
        } else {
            it
        }
    }.query("per_page", "100")
    return DataFrame.readJsonStr(client(request).bodyString())
}

In [31]:
val credentials: Credentials? = null

fun readDf(url: String) = readDf(url, credentials)

In [32]:
import org.http4k.core.HttpHandler

data class PageInfo(val body: DataFrame<*>, val pageUrls: List<String>)

fun parseLinkHeader(links: Pair<String, String?>): String? = links.second?.split(",")
        ?.map { it.split(";")
        ?.map { it.trim() } }
        ?.find { (_, rel) -> rel == "rel=\"last\"" }
        ?.first()
        ?.drop(1)?.dropLast(1)
        

val PAGE_REGEX = "page=(\\d+)$".toRegex()
fun readPageInfo(client: HttpHandler, url: String): PageInfo {
    val firstPage = client(Request(GET, url).query("per_page", "100"))
    val firstPageDf = DataFrame.readJsonStr(firstPage.bodyString())
    val links = firstPage.headers.find { (name, _) -> name == "Link" } ?: return PageInfo(firstPageDf, emptyList())
    val lastPageUrl = parseLinkHeader(links) ?: error("Expected 'Link' header with non empty value 'rel=last' attribute on endpoint $url, but was $links")
    val n = PAGE_REGEX.find(lastPageUrl)?.groupValues?.get(1)?.toInt()!!
    val pageUrls = (2..n).map {
        lastPageUrl.replace(PAGE_REGEX, "page=$it")    
    }
    return PageInfo(firstPageDf, pageUrls)
}

In [33]:
fun readPaginatedEndpoint(url: String): DataFrame<*> {
    val pageInfo = readPageInfo(client, url)
    return pageInfo.body.concat(pageInfo.pageUrls.map { readDf(it) }.concat())
}

Section 2. Build the data sample

In [34]:
val jb = readDf("https://api.github.com/orgs/JetBrains")
jb

DataFrame [1 x 29]

In [35]:
jb.schema()

login: String
id: Int
node_id: String
url: String
repos_url: String
events_url: String
hooks_url: String
issues_url: String
members_url: String
public_members_url: String
avatar_url: String
description: String
name: String
company: Any?
blog: String
location: String
email: Any?
twitter_username: String
is_verified: Boolean
has_organization_projects: Boolean
has_repository_projects: Boolean
public_repos: Int
public_gists: Int
followers: Int
following: Int
html_url: String
created_at: String
updated_at: String
type: String


Most of the data in the dataframe are just URLs. 

In [36]:
val jb1 = jb
    .add("repos") { readDf(repos_url) }
    .add("members") { readPaginatedEndpoint("${url}/public_members") }
jb1

DataFrame [1 x 31]

But now we have repositories, each repository has contributors_url. If we want to download contributors, we need to add a new column to `repos`. 

In [37]:
val jb2 = jb1
    .convert { repos }.with { 
        it.add("contributors") { readDf(contributors_url) }
    }

Unfortunately, it gets worse when you need to add a new child column to some deeply nested column. Like, for example, downloading a list of followers for each contributor of each repository would require writing this code:
```
jb2.convert { repos }.with {
    it.convert { 
        contributors 
    }.with { 
        it.add("followers") { readDf(followers_url) } 
    }
}
```
You can still do it, if you need to.

In [38]:
jb2.schema()

login: String
id: Int
node_id: String
url: String
repos_url: String
events_url: String
hooks_url: String
issues_url: String
members_url: String
public_members_url: String
avatar_url: String
description: String
name: String
company: Any?
blog: String
location: String
email: Any?
twitter_username: String
is_verified: Boolean
has_organization_projects: Boolean
has_repository_projects: Boolean
public_repos: Int
public_gists: Int
followers: Int
following: Int
html_url: String
created_at: String
updated_at: String
type: String
repos: *
    id: Int
    node_id: String
    name: String
    full_name: String
    private: Boolean
    owner:
        login: String
        id: Int
        node_id: String
        avatar_url: String
        gravatar_id: String
        url: String
        html_url: String
        followers_url: String
        following_url: String
        gists_url: String
        starred_url: String
        subscriptions_url: String
        organizations_url: String
        repos_url

Resulting dataframe has a lot of columns. We can use `select` and `remove` to filter them. Both operations provide DSL for selecting arbitrary set of columns https://kotlin.github.io/dataframe/columnselectors.html

In [39]:
val minifiedSample = jb2
    .select { cols(url, name, repos, members) }
    // Like before, we use convert to create dataframe with changed values in nested column
    .convert { members }.with { it.remove { endsWith("_url") } }
    .convert { repos }.with { it.remove { endsWith("_url").except(html_url) } }
    .convert { repos }.with { it.remove { owner } }
    .convert { repos }.with { 
        it.convert { contributors }.with { it.remove { endsWith("_url") } } 
    }

In [40]:
minifiedSample.schema()

url: String
name: String
repos: *
    id: Int
    node_id: String
    name: String
    full_name: String
    private: Boolean
    html_url: String
    description: String?
    fork: Boolean
    url: String
    created_at: String
    updated_at: String
    pushed_at: String
    homepage: String?
    size: Int
    stargazers_count: Int
    watchers_count: Int
    language: String?
    has_issues: Boolean
    has_projects: Boolean
    has_downloads: Boolean
    has_wiki: Boolean
    has_pages: Boolean
    forks_count: Int
    archived: Boolean
    disabled: Boolean
    open_issues_count: Int
    license:
        key: String?
        name: String?
        spdx_id: String?
        url: String?
        node_id: String?
    allow_forking: Boolean
    is_template: Boolean
    topics: List<String>
    visibility: String
    forks: Int
    open_issues: Int
    watchers: Int
    default_branch: String
    permissions:
        admin: Boolean
        maintain: Boolean
        push: Boolean
        

In [41]:
minifiedSample.writeJson("jetbrains.json")